In [1]:
import pandas as pd

top100 = pd.read_csv("HP/12weekTop100.csv")
clients = pd.read_csv("HP/clientsTotal.csv")




FileNotFoundError: [Errno 2] File b'HP/12weekTop100.csv' does not exist: b'HP/12weekTop100.csv'

### Rate: % of total Sale

In [ ]:
df = pd.merge(top100, clients, on="cod_cliente", how="left")
df["rate"] = df["Total_x"] / df["Total_y"] * 100
df = df.loc[:, ["cod_cliente", "cod_item", "rate"]]
df.head()

import numpy as np

dfStats = df.groupby("cod_item").agg({"rate": [np.size, np.mean]})



### At least 200 clients have bought the item

In [ ]:
mostPopular = dfStats["rate"]["size"] >= 100
dfStats[mostPopular].sort_values([("rate", "mean") ], ascending=False)[:15]


In [ ]:
# Ratings

itemsRating = df.pivot_table(index=['cod_cliente'],columns=['cod_item'],values='rate')
itemsRating.head()

#### Each sku correlated with others skus

In [ ]:
similarSkus = itemsRating.corrwith(itemsRating["11740"])
similarSkus = similarSkus.dropna()
newDf = pd.DataFrame(similarSkus)
newDf.head(10)

In [ ]:
itemsRating[["11740", "17175"]]

### Example of correlation between products

In [ ]:
import numpy as np
items1 = itemsRating["11740"]
items2 = itemsRating["17175"]
merge = pd.concat([items1, items2], axis=1)
merge = merge.dropna(axis=0)
test1= merge.loc[:, ["11740"]].values
test2 = merge.loc[:, ["17175"]].values
np.corrcoef( test1.ravel(), test2.ravel())

In [ ]:
df = dfStats[mostPopular].join(pd.DataFrame(similarSkus,  columns=['similarity']))

In [ ]:
df.head()

In [ ]:
df.sort_values(['similarity'], ascending=False)[:15]

### Second iteration

In [ ]:
# same principle
corrMatrix = itemsRating.corr(method='pearson', min_periods=100)
## print(corrMatrix.loc["11740", ["17175"] ])
corrMatrix.head()

In [ ]:
clientRatings = itemsRating.loc["HPC000111"].dropna()
clientRatings

In [ ]:
simCandidates = pd.Series()
# Iterate over the skus that client bought
for i in range(0, len(clientRatings.index)):
    # print ("Adding sims for " + clientRatings.index[i] + "...")
    # Similar skus
    sims = corrMatrix[clientRatings.index[i]].dropna()
    # Scaled to the "rating"
    sims = sims.map(lambda x: x * clientRatings[i])
    # Add the score to the list of similarity candidates
    simCandidates = simCandidates.append(sims)
    
#Glance at our results so far:
print ("sorting...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(10))

In [ ]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)

###### Similar SKUS of Client HPC000111 that have bought


In [ ]:
filteredSims = simCandidates[~simCandidates.index.isin(clientRatings.index)]
filteredSims.head(20)
